In [1]:
#load "Paket.fsx"
Paket.Package
  [ "QBitNinja.Client -Version 1.0.3.46"
    "NBitcoin -Version 4.0.0.48"
    "FSharp.Data -Version 2.4.6"
    "Newtonsoft.Json -Version 10.0.3"    
  ]    
#load "Paket.Generated.Refs.fsx"

open System
open NBitcoin
open NBitcoin.Crypto   
open NBitcoin.OpenAsset 
open NBitcoin.Stealth 
open QBitNinja.Client 
open QBitNinja.Client.Models 
open System.Collections.Generic 
open System.Linq
open System.Collections   

open FSharp.Data
open FSharp.Data.JsonExtensions

type secretJson = JsonProvider<"""{
  "secret": ""
}""">
      
let sj = secretJson.GetSample()

In [6]:
let alice = secretJson.Load(@"/home/nbuser/library/secret.json").JsonValue?secret.AsString()  
let bob = secretJson.Load(@"/home/nbuser/library/secret_dest.json").JsonValue?secret.AsString()  
let zorro = secretJson.Load(@"/home/nbuser/library/secret_zorro.json").JsonValue?secret.AsString()  
  
let happy = secretJson.Load(@"/home/nbuser/library/secret_zorro.json").JsonValue?secret.AsString() 

let alice_private  = Key.Parse(alice, Network.TestNet) 
let bob_private  = Key.Parse(bob, Network.TestNet) 
let zorro_private  = Key.Parse(zorro, Network.TestNet) 
let happy_private  = Key.Parse(zorro, Network.TestNet)  

In [7]:
let ZorroComp:Script = 
                PayToMultiSigTemplate
                            .Instance
                            .GenerateScriptPubKey(2,  [| alice_private.PubKey; bob_private.PubKey; zorro_private.PubKey |])

In [8]:
printfn "%A" ZorroComp

2 0364dfe2fa8c584d9af693fca129f9c031e4108be2b6d31ac3f6846ef1ba7ecdc1 0230621e0bd25d7207cacd09368b62b4107bca1a416f12d86e4f159e718e9d7750 02016e47a7061e49d9e0b13662e8c6efa0c6628051c11717ebb229f01231a0c5f0 3 OP_CHECKMULTISIG


In [12]:
let client:QBitNinjaClient  = new QBitNinjaClient(Network.TestNet)

let transactionId = uint256.Parse("d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b")

// Query the transaction
let transactionResponse:GetTransactionResponse  = client.GetTransaction(transactionId).Result

let transaction:NBitcoin.Transaction  = transactionResponse.Transaction

let receivedCoins = transactionResponse.ReceivedCoins;
let mutable myCoins:Coin = null 
for coin in receivedCoins do
    if (coin.TxOut.ScriptPubKey =  alice_private.ScriptPubKey) then
        myCoins<- coin:?>Coin
        
printfn "%A" myCoins.Amount

0.53990000


In [10]:
let mutable txBuilder = new TransactionBuilder() 
let mutable tx:Transaction = txBuilder
                                .AddCoins(myCoins) 
                                .AddKeys(alice_private)
                                .Send(ZorroComp, new Money(0.1m, MoneyUnit.BTC))
                                .SetChange(alice_private)
                                .SendFees(new Money(0.001m, MoneyUnit.BTC))
                                .BuildTransaction(true)


In [11]:
printfn "%A" tx

{
  "hash": "4c5994aa1b88ef449873bc590fa05ce215af4bffd6b9e68b8004e8bbb5f4418e",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 2,
  "lock_time": 0,
  "size": 305,
  "in": [
    {
      "prev_out": {
        "hash": "d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b",
        "n": 0
      },
      "scriptSig": "304402206d6078ea4ea2446cd8f01d7013a5904a6ce03e189a9f047394225df3abbb655a02203f998630ad81bfd12b9f52bdfd92bf20f7de999f3b1c85ed02f27ac3013518ab01 0364dfe2fa8c584d9af693fca129f9c031e4108be2b6d31ac3f6846ef1ba7ecdc1"
    }
  ],
  "out": [
    {
      "value": "0.43890000",
      "scriptPubKey": "OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "0.10000000",
      "scriptPubKey": "2 0364dfe2fa8c584d9af693fca129f9c031e4108be2b6d31ac3f6846ef1ba7ecdc1 0230621e0bd25d7207cacd09368b62b4107bca1a416f12d86e4f159e718e9d7750 02016e47a7061e49d9e0b13662e8c6efa0c6628051c11717ebb229f01231a0c5f0 3 OP_CHECKMULTISIG"
    }
  ]
}


In [9]:
let verify = txBuilder.Verify(tx)  //check fully signed
printfn "%A" (verify)

true


In [24]:
let mutable broadcastResponse:BroadcastResponse = client.Broadcast(tx).Result 


if not broadcastResponse.Success then
    printfn "ErrorCode: %A" (broadcastResponse.Error.ErrorCode) 
    printfn "Error message: %A" (broadcastResponse.Error.Reason) 
else
    printfn "Success!" 
    printfn "%A" (transaction.GetHash())

 

In [13]:
let transactionId' = uint256.Parse("d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b");
// Query the transaction
let transactionResponse' = client.GetTransaction(transactionId' ).Result 
let transaction'= transactionResponse'.Transaction

let receivedCoins' = transactionResponse.ReceivedCoins
let mutable myCoins':Coin = null 
for coin in receivedCoins' do
        myCoins<- coin:?>Coin
        
printfn "%A" myCoins.Amount


0.01000000


In [15]:
let mutable txBuilder'' = new TransactionBuilder();
let mutable tx' = txBuilder''
                    .AddCoins(myCoins)
                    .AddKeys(alice_private)
                    .Send(happy_private.PubKey, new Money(0.001m, MoneyUnit.BTC))
                    .SetChange(ZorroComp)
                    .SendFees(new Money(0.0001m, MoneyUnit.BTC))
                    .BuildTransaction(true)
        
printfn "%A" (txBuilder''.Verify(tx'))

false


In [16]:
printfn "%A" tx'

{
  "hash": "a52128348e348101940877b7b1c22f7b7865305cb8aa986f5fcd62080ef55bde",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 2,
  "lock_time": 0,
  "size": 209,
  "in": [
    {
      "prev_out": {
        "hash": "d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b",
        "n": 1
      },
      "scriptSig": ""
    }
  ],
  "out": [
    {
      "value": "0.00890000",
      "scriptPubKey": "2 0364dfe2fa8c584d9af693fca129f9c031e4108be2b6d31ac3f6846ef1ba7ecdc1 0230621e0bd25d7207cacd09368b62b4107bca1a416f12d86e4f159e718e9d7750 02016e47a7061e49d9e0b13662e8c6efa0c6628051c11717ebb229f01231a0c5f0 3 OP_CHECKMULTISIG"
    },
    {
      "value": "0.00100000",
      "scriptPubKey": "02016e47a7061e49d9e0b13662e8c6efa0c6628051c11717ebb229f01231a0c5f0 OP_CHECKSIG"
    }
  ]
}


In [17]:
let mutable txBuilder''' = new TransactionBuilder();
let mutable tx'' = txBuilder'''
                        .AddCoins(myCoins)
                        .AddKeys(alice_private, bob_private)
                        .Send(happy_private.PubKey, new Money(0.001m, MoneyUnit.BTC))
                        .SetChange(ZorroComp)
                        .SendFees(new Money(0.0001m, MoneyUnit.BTC))
                        .BuildTransaction(true)
        
printfn "%A" (txBuilder''.Verify(tx''))

true


In [18]:
printfn "%A" tx''

{
  "hash": "0ab5eb245121c867449ac81d33814a4e91ff15a223d7ea0d0fb64dd7ba498279",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 2,
  "lock_time": 0,
  "size": 315,
  "in": [
    {
      "prev_out": {
        "hash": "d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b",
        "n": 1
      },
      "scriptSig": "304402202a087611eb257aa90f78f37d000c491d09d2c592374c0d05071c019298ab51140220295d2d503c6505e553f2dc8987e7ea4e043e6105a299344bdf1662588088ba5c01 0230621e0bd25d7207cacd09368b62b4107bca1a416f12d86e4f159e718e9d7750"
    }
  ],
  "out": [
    {
      "value": "0.00890000",
      "scriptPubKey": "2 0364dfe2fa8c584d9af693fca129f9c031e4108be2b6d31ac3f6846ef1ba7ecdc1 0230621e0bd25d7207cacd09368b62b4107bca1a416f12d86e4f159e718e9d7750 02016e47a7061e49d9e0b13662e8c6efa0c6628051c11717ebb229f01231a0c5f0 3 OP_CHECKMULTISIG"
    },
    {
      "value": "0.00100000",
      "scriptPubKey": "02016e47a7061e49d9e0b13662e8c6efa0c6628051c11717ebb229f01231a0c5f0 OP_CHECKSIG"
    }
  ]
}


In [18]:
open Newtonsoft.Json
let jtx = JsonConvert.SerializeObject(txBuilder'')
printfn "%A" jtx

"{"CoinSelector":{"GroupByScriptPubKey":true},"DustPrevention":true,"FilterUneconomicalCoins":true,"FilterUneconomicalCoinsRate":null,"CoinFinder":null,"KeyFinder":null,"StandardTransactionPolicy":{"MaxTransactionSize":100000,"MaxTxFee":{"FeePerK":{"Satoshi":10000000}},"MinRelayTxFee":{"FeePerK":{"Satoshi":1000}},"ScriptVerify":32735,"CheckMalleabilitySafe":false,"CheckFee":true,"UseConsensusLib":false,"CheckScriptPubKey":true},"Extensions":[{},{},{},{}]}"


In [19]:
tx'.Sign(bob_private,myCoins )

In [20]:
printfn "%A" tx'

{
  "hash": "0ab5eb245121c867449ac81d33814a4e91ff15a223d7ea0d0fb64dd7ba498279",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 2,
  "lock_time": 0,
  "size": 315,
  "in": [
    {
      "prev_out": {
        "hash": "d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b",
        "n": 1
      },
      "scriptSig": "304402202a087611eb257aa90f78f37d000c491d09d2c592374c0d05071c019298ab51140220295d2d503c6505e553f2dc8987e7ea4e043e6105a299344bdf1662588088ba5c01 0230621e0bd25d7207cacd09368b62b4107bca1a416f12d86e4f159e718e9d7750"
    }
  ],
  "out": [
    {
      "value": "0.00890000",
      "scriptPubKey": "2 0364dfe2fa8c584d9af693fca129f9c031e4108be2b6d31ac3f6846ef1ba7ecdc1 0230621e0bd25d7207cacd09368b62b4107bca1a416f12d86e4f159e718e9d7750 02016e47a7061e49d9e0b13662e8c6efa0c6628051c11717ebb229f01231a0c5f0 3 OP_CHECKMULTISIG"
    },
    {
      "value": "0.00100000",
      "scriptPubKey": "02016e47a7061e49d9e0b13662e8c6efa0c6628051c11717ebb229f01231a0c5f0 OP_CHECKSIG"
    }
  ]
}
